<a href="https://colab.research.google.com/github/blinders79/product-search/blob/main/ProductSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade openai httpx

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.12.0
    Uninstalling h11-0.12.0:
      Successfully uninstalled h11-0.12.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.15.0
    Uninstalling httpcore-0.15.0:
      Successfully uninstalled httpcore-0.15.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.23.0
    Uninstalling httpx-0.23.0:
      Successfully uninstalled httpx-0.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.7.0 requires

In [2]:
!pip install --upgrade openai
!pip install httpx==0.23.0 # Downgrade httpx to a compatible version

import pandas as pd
import numpy as np
import os
from openai import OpenAI
from dotenv import load_dotenv

from sklearn.metrics.pairwise import cosine_similarity

# Loading API key and organization ID from a dotenv file
load_dotenv(dotenv_path='apikey.env.txt')

# Retrieving API key and organization ID from environment variables
APIKEY = os.getenv("APIKEY")


# Set your API key
client = OpenAI(api_key=APIKEY)

# Make a request to the GPT-4 model
#The openai.ChatCompletion.create method has been replaced with client.chat.completions.create
# in the latest version of the openai library.


  Using cached httpx-0.23.0-py3-none-any.whl.metadata (52 kB)
  Using cached httpcore-0.15.0-py3-none-any.whl.metadata (15 kB)
  Using cached h11-0.12.0-py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.23.0-py3-none-any.whl (84 kB)
Using cached httpcore-0.15.0-py3-none-any.whl (68 kB)
Using cached h11-0.12.0-py3-none-any.whl (54 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.7
    Uninstalling httpcore-1.0.7:
      Successfully uninstalled httpcore-1.0.7
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.7.0 requires a

In [3]:
products_data = pd.read_csv("/content/products_dataset.csv")

In [4]:
products_data["item"] = products_data['title'] + products_data['description']

In [5]:
response = client.embeddings.create(
    input=products_data.item.to_list(),
    model="text-embedding-3-small",
    dimensions=512
)
vector_embeddings = [d.embedding for d in response.data]
products_data['embedding_vector'] = vector_embeddings

In [6]:
user_prompt = "i'm remodelling my bathroom an need a stylish white sink with a facesut included"

In [7]:
response = client.embeddings.create(
    input=[user_prompt],
    model="text-embedding-3-small",
    dimensions=512
)
user_prompt_vector = [d.embedding for d in response.data]
print(user_prompt_vector[:10])
#

[[-0.010992810130119324, -0.04152626916766167, 0.030122781172394753, -0.04041839390993118, -0.018031645566225052, -0.06372200697660446, 0.0296261478215456, 0.033350903540849686, -0.02607330121099949, -0.057685986161231995, 0.023914853110909462, -0.03273966163396835, -0.04702745005488396, -0.028441864997148514, 0.053063467144966125, 0.1136910542845726, 0.0036674535367637873, -0.030753126367926598, -0.028289055451750755, 0.04710385575890541, -0.01203383132815361, 0.027945231646299362, -0.028078939765691757, -0.02733398973941803, 0.012492263689637184, 0.035031821578741074, 0.004479259718209505, -0.0323767364025116, 0.0011938324896618724, 0.009120880626142025, -0.05887026712298393, -0.05046568810939789, -0.036502622067928314, -0.020514817908406258, -0.0064228191040456295, 0.03854646533727646, 0.030504807829856873, -0.0027291011065244675, -0.022577760741114616, -0.03636891394853592, 0.010992810130119324, 0.03466889634728432, 0.05646350234746933, -0.022023823112249374, 0.02783062309026718, 0

In [9]:
product_vectors = [np.array(vec) for vec in products_data['embedding_vector']]
user_prompt_vector = np.array(user_prompt_vector)
similarity_matrix = cosine_similarity(product_vectors,user_prompt_vector)
similarity_values = [v[0] for v in similarity_matrix]
products_data['similarity_values'] = similarity_values
products_data.sort_values(by='similarity_values', ascending=False, inplace=True)
products_data.head(3)

,product_id,title,description,item,embedding_vector,similarity_values
79,P79,24.38 in. x 14 in. x 6 in. White Above Counter...,Bathroom vessel sinks make the bathroom look m...,24.38 in. x 14 in. x 6 in. White Above Counter...,"[-0.0011324023362249136, 0.0246982891112566, 0...",0.594462
983,P983,Decorum Vitreous China Wall-Hung Rectangle Ves...,The Decorum Wall-Hung Bathroom Sink from Ameri...,Decorum Vitreous China Wall-Hung Rectangle Ves...,"[-0.009413822554051876, -0.015025733970105648,...",0.586464
1638,P1638,Glass Vessel Sink in Smoky Black with Waterfal...,This flower petal formed vessel features grace...,Glass Vessel Sink in Smoky Black with Waterfal...,"[-0.0013510264689102769, 0.04279814660549164, ...",0.584534
